In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.constants import k
import sdr_utils as sutil
import adi
from matplotlib.widgets import Slider, Button

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
## SDR Setup
sample_rate = 1e6 # Hz
center_freq = 915e6 # Hz
num_samps = 100000 # number of samples per call to rx()

sdr = adi.Pluto("ip:192.168.2.1")
sdr.sample_rate = int(sample_rate)

# Config Tx
sdr.tx_rf_bandwidth = int(sample_rate) # filter cutoff, just set it to the same as sample rate
sdr.tx_lo = int(center_freq)
sdr.tx_hardwaregain_chan0 = -50 # Increase to increase tx power, valid range is -90 to 0 dB

# Config Rx
sdr.rx_lo = int(center_freq)
sdr.rx_rf_bandwidth = int(sample_rate)
sdr.rx_buffer_size = num_samps
sdr.gain_control_mode_chan0 = 'manual'
sdr.rx_hardwaregain_chan0 = 0.0 # dB, increase to increase the receive gain, but be careful not to saturate the ADC


In [4]:
## Using quick utilities:
bin_msg = np.array(sutil.binaryify("Hello World"))
barker_code = np.array([1,1,1,0,0,0,1,0,0,1,0]) # binary barker code
bin_msg = np.concatenate([barker_code, bin_msg])
print(f"Binary message being sent: {bin_msg}")
barker_train = sutil.pulse_train(barker_code)
train = sutil.pulse_train(bin_msg)
shaped = sutil.raised_cos_filter(train, β = 0.35)

I = shaped
Q = np.zeros(len(shaped))
IQ_send = I + 1j*Q
IQ_send 
samples = np.repeat(IQ_send, 16) # 16 samples per symbol (rectangular pulses)
samples *= 2**14 # The PlutoSDR expects samples to be between -2^14 and +2^14, not -1 and +1 like some SDRs
print(f"sample length {len(samples)}")

Binary message being sent: [1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0.]
sample length 12736


In [6]:
# Start the transmitter
sdr.tx_cyclic_buffer = True # Enable cyclic buffers
sdr.tx(samples) # start transmitting

sdr.rx_annotated = False

# Clear buffer just to be safe
for i in range (0, 10):
    raw_data = sdr.rx()

# Receive samples
rx_samples = sdr.rx()
print(f"rx sample size raw: {len(rx_samples)}")
# Stop transmitting
sdr.tx_destroy_buffer()

print(rx_samples)

rx sample size raw: 100000
[0.+0.j 0.+0.j 0.+1.j ... 0.+0.j 0.+0.j 0.+0.j]


In [12]:
print(sdr.rx_channel_names)
print(sdr.dds_scales)
phy = sdr.ctx.find_device("ad9361-phy")
# Read product ID register
pi = phy.reg_read(0x37)
print(f"ID: {hex(pi)}")
# Enable near-end loopback in the HDL core
rxfpga = sdr.ctx.find_device("cf-ad9361-lpc")
rxfpga.reg_write(0x80000418, 0x1)  # I channel
rxfpga.reg_write(0x80000458, 0x1)  # Q channel

for dattr in phy.debug_attrs:
    print(dattr, phy.debug_attrs[dattr])
adi.
print(sdr.multichip_sync)

['voltage0', 'voltage1']
['0.000000', '0.000000', '0.000000', '0.000000']
ID: 0xa
digital_tune digital_tune
calibration_switch_control calibration_switch_control
multichip_sync multichip_sync
gaininfo_rx2 gaininfo_rx2
gaininfo_rx1 gaininfo_rx1
bist_timing_analysis bist_timing_analysis
gpo_set gpo_set
bist_tone bist_tone
bist_prbs bist_prbs
loopback loopback
initialize initialize
adi,bb-clk-change-dig-tune-enable adi,bb-clk-change-dig-tune-enable
adi,axi-half-dac-rate-enable adi,axi-half-dac-rate-enable
adi,txmon-2-lo-cm adi,txmon-2-lo-cm
adi,txmon-1-lo-cm adi,txmon-1-lo-cm
adi,txmon-2-front-end-gain adi,txmon-2-front-end-gain
adi,txmon-1-front-end-gain adi,txmon-1-front-end-gain
adi,txmon-duration adi,txmon-duration
adi,txmon-delay adi,txmon-delay
adi,txmon-one-shot-mode-enable adi,txmon-one-shot-mode-enable
adi,txmon-dc-tracking-enable adi,txmon-dc-tracking-enable
adi,txmon-high-gain adi,txmon-high-gain
adi,txmon-low-gain adi,txmon-low-gain
adi,txmon-low-high-thresh adi,txmon-low-high

AttributeError: 'Pluto' object has no attribute 'multichip_sync'

In [ ]:
# plt.plot(np.real(IQ_send), '.-', zorder = 1)
# plt.plot(np.imag(IQ_send), '.-', zorder = 2)
# plt.legend(["I", "Q"])
# plt.title("Original Hello World Signal")
# plt.show()

# filter rx with root raised:
rx_samples = sutil.raised_cos_filter(rx_samples)

### Clock recovery -- important for good data later
rx_samples_cr = sutil.clock_recovery(np.copy(rx_samples))
 
# rx_samples = sutil.phase_detector_2(rx_samples)

print(f"rx sample size: {len(rx_samples)}")

# np.set_printoptions(threshold=np.inf)

# bin_rx = np.array(sutil.simple_ecode_recv(rx_samples))
# contains_bin = np.isin(bin_msg, bin_rx)
# print(f"Message contains binary string: {contains_bin}")
# print(f"binary message: {bin_msg}")
# print(f"binary receive: {bin_rx[-1*len(bin_msg):]}")
# bin_conv = np.convolve(bin_rx, bin_msg)
# plt.plot(bin_rx)
# plt.show()



tc = np.arange(0,2*np.pi,1e-2)
plt.plot(np.cos(tc), np.sin(tc), linewidth=1, color="grey")
plt.plot(np.real(rx_samples), np.imag(rx_samples), '.', color="red")
plt.title("IQ Plot of BPSK Message Raw")
plt.grid(True)
plt.show()

plt.plot(np.cos(tc), np.sin(tc), linewidth=1, color="grey")
plt.plot(np.real(rx_samples_cr), np.imag(rx_samples_cr), '.', color="red")
plt.title("IQ Plot of BPSK Message with Clock Recovery")
plt.grid(True)
plt.show()

In [ ]:
## This block of code allows the researcher to manually adjust a frequency
## shift on the received data from the PlutoSDR

IQ_no_shift = np.copy(rx_samples)
def plot_fo(Δf):
    global sample_rate, IQ_no_shift
    fs = sample_rate # assume our sample rate is 1 MHz
    fo = Δf #13000 # simulate freq offset
    Ts = 1/fs # calc sample period
    # Trying to add equal frequency offset to all samples
    
    t = np.arange(0, Ts*len(IQ_no_shift), Ts) # create time vector
    t_comp = np.exp(1j*2*np.pi*Δf)#*t)
    IQ_shift = np.copy(IQ_no_shift) * t_comp # perform freq shift
    #IQ_shift = sutil.add_frequency_offset(IQ_no_shift, fs=fs, Δf=fo)
    
    plt.plot(np.real(IQ_shift), '.-', zorder = 1)
    plt.plot(np.imag(IQ_shift), '.-', zorder = 2)
    plt.legend(["I", "Q"])
    plt.show()
    
    tc = np.arange(0,2*np.pi,1e-2)
    plt.plot(np.cos(tc), np.sin(tc), linewidth=1, color="grey")
    plt.plot(np.real(IQ_shift), np.imag(IQ_shift), '.', color="red")
    plt.title("IQ Plot of Shifted Quadrature")
    plt.grid(True)
#     plt.xlim([-1.2, 1.2])
#     plt.ylim([-1.2, 1.2])
    plt.show()
    
    plt.plot(np.cos(tc), np.sin(tc), linewidth=1, color="grey")
    plt.plot(np.real(t_comp), np.imag(t_comp), '.', color="red")
    plt.title("Shifting Plot")
    plt.grid(True)
#     plt.xlim([-1.2, 1.2])
#     plt.ylim([-1.2, 1.2])
    plt.show()
    print(f"Length of vector: {len(IQ_shift)}")
interact(plot_fo, Δf = (-10, 10, 0.01)); # -0.22

In [ ]:
## sweep for minimizing Quadrature with frequncy shift
def cost(IQ):
    return np.sum(np.abs(np.imag(IQ)))
def f_shift(IQ, θ):
    return np.copy(IQ) * np.exp(1j*θ) # perform freq shift

print(cost(IQ_no_shift))
print(cost(f_shift(IQ_no_shift, 0.05)))
costs = []
angles = np.arange(0,2*np.pi,0.0001)
for angle in angles:
    costs.append(cost(f_shift(IQ_no_shift, angle)))
    
costs = np.array(costs)
plt.plot(angles, costs)
plt.show()

In [ ]:
θ_min = angles[np.argmin(costs)]
print(f"Minimum θ: {θ_min}")

IQ_opt_shift = f_shift(IQ_no_shift, θ_min)
sutil.plot_IQ(plt, IQ_opt_shift, "Optimally Shifted θ to minimize Quadrature")
sutil.plot_IQ_samples(plt, IQ_opt_shift, "Optimally shifted θ RX samples")

# note: may normalize data in the future

In [ ]:
# perform clock recovery on optimally shifted signal
IQ_MMCR = sutil.clock_recovery(np.copy(IQ_opt_shift))
sutil.plot_IQ(plt, IQ_opt_shift, "Optimally Shifted θ to minimize Quadrature")
sutil.plot_IQ(plt, IQ_MMCR, "Optimal shift with clock recovery")


sutil.plot_IQ_samples(plt, IQ_opt_shift[1000:2000], "Optimally shifted θ RX samples")
sutil.plot_IQ_samples(plt, IQ_MMCR[1000:2000], "Optimally shifted θ RX samples with MMCR")

print(f"Length of original: {len(IQ_opt_shift)}")
print(f"Length of MMCR: {len(IQ_MMCR)}")

In [ ]:
def delay_plot(Δsample, N_taps):
    global IQ_no_shift
    delay = Δsample # fractional delay, in samples
    N = N_taps #21 # number of taps
    IQ_delay = sutil.add_delay(IQ_no_shift, Δsample=delay, N=N)

#     plt.plot(m_shaped, zorder = 1)
#     plt.plot(m_shaped_delay, zorder = 2)
#     plt.legend(["Original", "Delayed"])
#     plt.show()
    tc = np.arange(0,2*np.pi,1e-2)
    plt.plot(np.cos(tc), np.sin(tc), linewidth=1, color="grey")
    plt.plot(np.real(IQ_delay), np.imag(IQ_delay), '.', color="red")
    plt.title("IQ Plot of Delayed Quadrature")
    plt.grid(True)
#     plt.xlim([-1.2, 1.2])
#     plt.ylim([-1.2, 1.2])
    plt.show()
    
interact(delay_plot, Δsample=(-1,1,0.01), N_taps=(1,100,1));

In [ ]:
t_comp = np.exp(1j*2*np.pi*-0.08) #*t) <-- removed time compensation component
IQ_shift = np.copy(IQ_no_shift) * t_comp # perform freq shift

sutil.plot_IQ(plt, IQ_shift, "Shifted IQ")

IQ_shift_filtered = sutil.raised_cos_filter(IQ_shift)
sutil.plot_IQ(plt, IQ_shift_filtered, "Filtered IQ")